In [1]:
from code_analyzer_agent import CodeAnalyzerAgent, create_code_analysis_tools

# Configuration
CONTEXT_START = "context/v1/start-here.md"
APP_NAME = "cbv"
PATTERN = "ucc"
SOURCE_CODE = "source-code"

# Create agent
agent = CodeAnalyzerAgent(
    context_start_path=CONTEXT_START,
    app_name=APP_NAME,
    pattern=PATTERN,
    source_code_path=SOURCE_CODE,
    base_url="http://localhost:1234/v1",
    model="qwen/qwen3-coder-30b",
    log_dir="logs"
)

# Register tools
tools, schemas = create_code_analysis_tools(
    source_code_path=SOURCE_CODE,
    context_dir=agent.context_dir
)

for tool_name, tool_func, schema in zip(tools.keys(), tools.values(), schemas):
    agent.register_tool(tool_name, tool_func, schema)

print("✅ Agent ready")

📁 Session logging enabled: /Users/Khaled.Alabsi/projects/llm-agent/logs/session_20251111_190500
   Tool logs: logs/session_20251111_190500/tool
   LLM logs: logs/session_20251111_190500/llm

🔍 Code Analyzer Agent Initialized
   App Name: cbv
   Pattern: ucc
   Source Code: source-code
   Context: context/v1/start-here.md
✓ Registered tool: read_file
✓ Registered tool: search_files
✓ Registered tool: grep_code
✓ Registered tool: write_analysis
✅ Agent ready


In [ ]:
# Run the agent
agent.reset_conversation()

task = f"""
  You are analyzing the '{APP_NAME}' application using the {PATTERN.upper()} pattern.

  INSTRUCTIONS - Execute these steps in order:

  1. READ the context file 'start-here.md' to understand Phase 1, Step 1
  2. SEARCH the source code for REST controllers:
     - Use search_files with pattern "**/*Controller.java" in search_in="source"
     - Look in base_path like "ucc/cbv" or similar
  3. SEARCH the source code for LSG pages:
     - Use search_files with pattern "**/*Page.java"
  4. READ at least 2-3 of the files you found to understand what they do
  5. SUMMARIZE your findings with specific file paths and purposes

  DO NOT just explain what to do - ACTUALLY USE YOUR TOOLS to search and analyze the code.
  Start by reading start-here.md, then immediately search for controllers.
  """

response = agent.run(task, max_iterations=20)

print("\n" + "="*80)
print("RESPONSE:")
print("="*80)
print(response)

🔄 Conversation reset

🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 
CODE ANALYZER RUN STARTED: 2025-11-11 19:05:00
🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 

────────────────────────────────────────────────────────────────────────────────
ITERATION 1/20
────────────────────────────────────────────────────────────────────────────────

📤 SENDING TO LLM
Messages: 1
Tools available: 4
📥 LLM Response received
💾 Logged to: logs/session_20251111_190500/llm/001_request_response.json

🔧 Tool calls requested: 1
   - read_file

🔧 Executing: read_file
   Args: {
  "file_path": "start-here.md"
}...
   ✓ Executed successfully
💾 Tool logged to: logs/session_20251111_190500/tool/002_read_file.log

────────────────────────────────────────────────────────────────────────────────
ITERATION 2/20
────────────────────────────────────────────────────────────────────────────────

📤 SENDING TO LLM
Messages: 3
Tools available: 4
📥 LLM Res